In [5]:
import src.utils.regime as regime
import src.scanner as scanner
import src.floor_ceiling_regime as sfcr
import json

In [1]:

class DataLoader:
    def __init__(self, data_file_path):
        with open(data_file_path, 'r') as data_fp:
            self._data = json.load(data_fp)

    @property
    def paths(self):
        return self._data['paths']

    @property
    def base(self):
        return self.paths['base']

    def history(self, bench: str, interval: int):
        return self.paths['history'][str(interval)][bench]

    def bench(self, bench, interval):
        return self.paths['bench'][str(interval)][bench]


def main_re_download_data():
    """re download stock and bench data, write to locations specified in paths.json"""
    sp500_wiki = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    ticks, _ = scanner.get_wikipedia_stocks(sp500_wiki)
    data_loader = DataLoader('paths.json')
    bench = 'SPY'
    days = 59
    interval = 15
    interval_str=f'{interval}m'
    history_path=data_loader.history(bench=bench, interval=interval)
    bench_path=data_loader.bench(bench=bench, interval=interval)
    downloaded_data = scanner.yf_download_data(ticks, days, interval_str)
    bench_data = scanner.yf_get_stock_data('SPY', days, interval_str)
    relative = regime.simple_relative(downloaded_data, bench_data.close)
    # TODO add relative data to schema
    relative.to_csv(f'{data_loader.base}\\spy_relative_history_{interval_str}.csv')
    downloaded_data.to_csv(history_path)
    bench_data.to_csv(bench_path)


def run_backtest():
    _sp500_wiki = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    ticks_, _ = scanner.get_wikipedia_stocks(_sp500_wiki)
    _days = 59
    _interval = 15
    _interval_str = f'{_interval}m'

    scanner_ = scanner.StockDataGetter(
        data_getter_method=lambda s: scanner.yf_get_stock_data(s, days=59, interval=_interval_str),
    ).yield_strategy_data(
        bench_symbol="SPY",
        symbols=ticks_,
        # symbols=['ABBV'],
        strategy=lambda pdf_, bdf_: (
            sfcr.fc_scale_strategy(
                price_data=scanner.data_to_relative(pdf_, bdf_),
                distance_pct=0.05,
                retrace_pct=0.05,
                swing_window=63,
                sw_lvl=3,
                regime_threshold=0.5,
                trail_offset_pct=0.01,
                r_multiplier=1.5,
                entry_lvls=None,
                highest_peak_lvl=3,
            )
        ),
        expected_exceptions=(regime.NotEnoughDataError, sfcr.NoEntriesError)
    )
    stat_overview_ = scanner.run_scanner(
        scanner=scanner_,
        stat_calculator=lambda data_, entry_signals_: sfcr.calc_stats(
            data_,
            entry_signals_,
            min_periods=50,
            window=200,
            percentile=0.05,
            limit=5,
            freq=f'{_interval}T',
            # freq='1D',
            # freq='5T',
        ),
        relative_side_only=True
    )
    # stat_overview_ = stat_overview_.sort_values('risk_adj_returns_roll', axis=1, ascending=False)
    stat_overview_.to_csv('stat_overview.csv')
    print('done')

IndentationError: expected an indented block (863730350.py, line 5)

In [6]:
# TODO concat all strategy data and write to feather